In [1]:
%matplotlib inline


import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

import json


from datetime import datetime, date



In [2]:

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Load the world map

In [3]:
# %%writeandexecute -i identifier load.py

def loadGeopandas():    
    shapefile = 'data/countries_110m/ne_110m_admin_0_countries.shp'
    gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
    gdf.head()
    gdf.columns = ['country', 'country_code', 'geometry']
    gdf.head()
    
    print(gdf[gdf['country'] == 'Antarctica'])
    #Drop row corresponding to 'Antarctica'
    gdf = gdf.drop(gdf.index[159])
    return gdf

In [4]:
gdf = loadGeopandas()
gdf.head()

        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  


,country,country_code,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


# Get data from CSSEGISandData

In [5]:
def getTimeSeries(fname):
    print(fname)
    df = pd.read_csv(fname)
    if not fname.count('web-data'):
        countryData = df
        return countryData.drop(['Province/State', 'Lat', 'Long'], axis=1)
    return df
    
def fetchCountryData(freq='1d'):
    gisandata ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_'
    confirmed = getTimeSeries(f'{gisandata}confirmed_global.csv')

    return confirmed
   

In [6]:
country_df = getTimeSeries('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
country_df.columns = map(str.lower, country_df.columns)


data = fetchCountryData()
data = data.rename(columns={'Country/Region': 'country_region'})


https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv


$\textbf{Merge the covid data with the world map}$

In [7]:
# Get csv with the countries and alpha codes
countries_code = pd.read_csv('JohnsHopkins-to-A3.csv')
countries_code.head()

# ToDO: merge to country_df according to country field
countries_code = countries_code.rename(columns={'Country/Region': 'country_region'})
countries_code = countries_code.rename(columns={'alpha3': 'country_code'})


# View the change in the world by using the slide bar

In [8]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper,LogColorMapper, ColorBar, LogTicker
from bokeh.palettes import brewer, gray, Turbo256, inferno

from bokeh.io import curdoc
from bokeh.models import Slider, HoverTool, Column
from bokeh.layouts import widgetbox, row, column

#Input GeoJSON source that contains features for plotting.



In [9]:

def json_bydate(dateval):
    #provide date in format m/dd/yy as str
    
    tmp_df = data[['country_region', dateval]].copy()
    tmp_df = tmp_df.rename(columns={dateval: 'confirmed'})
    
    max_conf_bydate =max( tmp_df['confirmed'])

    print('max value on ', dateval, max_conf_bydate)
    tmp_df_code = pd.merge(countries_code, tmp_df, on='country_region')
    
    merged = gdf.merge(tmp_df_code, on = 'country_code', how = 'left')
    
    
    
    merged_json = json.loads(merged.to_json())
    
    json_data = json.dumps(merged_json)
    
    return json_data, max_conf_bydate


    

In [10]:
palette = inferno(255)[::-1]

In [11]:
def get_strdate(val):
    
    mounth = val.strftime("%m")
    day = slider_def.strftime("%d")
    yer = slider_def.strftime("%y")
    
    if mounth[0] == '0':
        mounth = mounth[1]

    if day[0] == '0':
        day = day[1]
    return '/'.join([mounth,day, yer])

In [14]:
hover = HoverTool(tooltips = [ ('Country/region','@country'),('Covid', '')])

slider_def = date(2020, 3, 15)
dstr = get_strdate(slider_def)

p_slider = figure(title = f'Confirmed COVID-19 on {dstr}', plot_height = 500 , plot_width = 950)
p_slider.xgrid.grid_line_color = None
p_slider.ygrid.grid_line_color = None
#Add patch renderer to figure. 


geojs, maxconf = json_bydate(dstr)
geosource_date= GeoJSONDataSource(geojson = geojs)


color_mapper_date = LogColorMapper(palette = palette, low = 1, high = maxconf, nan_color = '#d9d9d9')

color_bar_date = ColorBar(color_mapper=color_mapper_date, ticker = LogTicker(),  label_standoff=12, 
                          border_line_color=None, location=(0,0))

p_slider.patches('xs','ys', source = geosource_date,fill_color = {'field' :'confirmed', 'transform' : color_mapper_date},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)


p_slider.add_layout(color_bar_date, 'right')




max value on  3/15/20 67794


In [15]:
def update_plot(attr, old, new):
    f.write("Update func")
    
    dateval = new
    f.write(dateval)
    p_slider.title.text = f'Confirmed COVID-19 on {dateval}'
    dstr = get_strdate(dateval)
    new_data = json_bydate(dstr)
    print(new_data)
    geosource_date.geojson = new_data
    p_slider.patches('xs','ys', source = geosource_date,fill_color = {'field' :'confirmed', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
    
    layout = column(p_slider,widgetbox(slider))
    
    show(layout)

   

    

In [17]:
from bokeh.models.widgets import DateSlider
from bokeh.models import CustomJS

print(slider_def)
slider = DateSlider(title="Date Range: ", start=date(2020, 1, 22),
                    end=date.today(), value=slider_def)
layout = column(p_slider,widgetbox(slider))



2020-03-15


In [ ]:
    
curdoc().add_root(layout)
# curdoc().title = "DateRangeSlider Example"

#Display plot inline in Jupyter notebook
# output_notebook()
#Display plot
# curdoc().add_root(row(step_select, slider))

show(layout)
f.write(str(slider.value))
